# Test and Demonstrate zero_gpu_vram_usage Analysis

This notebook tests and demonstrates the main functions in `src/analysis/zero_gpu_vram_usage.py` for analyzing hybrid workload efficiency and CPU-GPU balance.

In [ ]:
# Import required modules
import sys
from pathlib import Path

# Add src to sys.path for module imports
sys.path.append(str(Path.cwd().parent / "src"))

In [ ]:
project_root = str(Path.cwd().resolve().parent)
print(f"Project root: {project_root}")

In [ ]:
# Add project root to sys.path for module imports
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.analysis.vram_usage import EfficiencyAnalysis

# Automatically reload modules before executing code
# This is useful for development to see changes without restarting the kernel.
%load_ext autoreload
# Reload all modules imported with %aimport every time before executing the Python code typed.
%autoreload 1
%aimport src.analysis.vram_usage

In [ ]:
from src.database.DatabaseConnection import DatabaseConnection

# Connect to the database
db = DatabaseConnection(db_url="../slurm_data_new.db")

# Query jobs with GPUs
gpu_df = db.connection.query("SELECT * FROM Jobs WHERE GPUs > 0").to_df()

In [ ]:
# Initialize the EfficiencyAnalysis class
analysis = EfficiencyAnalysis(df=gpu_df)

# Load jobs data
jobs_df = analysis.jobs_df

# Display the loaded data
display(jobs_df.head())

In [ ]:
import pandas as pd
multi = gpu_df[gpu_df["GPUs"] > 1]
display(jobs_df["StartTime"].head(10))
display(jobs_df["StartTime"].tail(10))


In [ ]:
# Calculate efficiency metrics
from src.preprocess.preprocess import preprocess_data

preprocessed_df = preprocess_data(jobs_df)

analysis.jobs_df = preprocessed_df

filtered_jobs = analysis.calculate_efficiency_metrics(
    vram_constraint_filter=0, allocated_vram_greater_than=0, gpu_mem_usage_min=0, gpus_min=1, elapsed_seconds_min=600
)

# Display the filtered jobs
display(filtered_jobs.head())

In [ ]:
# Run evaluate_cpu_gpu_balance
analysis_results = analysis.evaluate_cpu_gpu_usage()

# Display key summary statistics
print("Total jobs:", analysis_results["total_jobs"])
print("Total GPU hours:", analysis_results["total_gpu_hours"])
print("Average VRAM efficiency:", f"{analysis_results['avg_efficiency']:.2%}")
print("Median VRAM efficiency:", f"{analysis_results['median_efficiency']:.2%}")

# Show recommendations
print("\nRecommendations:")
for rec in analysis_results["report"]:
    print("-", rec)
# Display efficiency patterns table
analysis_results["efficiency_patterns"]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot VRAM efficiency distribution
plt.figure(figsize=(8, 5))
sns.histplot(filtered_jobs["vram_efficiency"].dropna(), bins=30, kde=True)
plt.xlabel("VRAM Efficiency")
plt.ylabel("Number of Jobs")
plt.title("Distribution of VRAM Efficiency for Hybrid Jobs")
plt.xlim(0, 1.0)
plt.show()

# print(analysis_results['top_inefficient_users'].head(5))

# Plot top inefficient users by GPU hours
if "top_inefficient_users" in analysis_results:
    top_users = analysis_results["top_inefficient_users"].head(10)
    plt.figure(figsize=(10, 5))
    sns.barplot(y=top_users.index, x=top_users["GPU_Hours"], orient="h")
    plt.xlabel("GPU Hours")
    plt.ylabel("User")
    plt.title("Top 10 Inefficient Users by GPU Hours (Hybrid Jobs)")
    plt.show()